目錄
* [資料（內文）處理](#資料（內文）處理)
    * [正規表示法](#正規表示法)
    * [斷詞](#斷詞)
    * [匯出資料](#匯出資料)
* [訓練模型（分類器）](#訓練模型（分類器）)
    * [Count-Vector](#Count-Vector)
    * [Tfidf-Vector](#Tfidf-Vector)
    * [統整分類結果](#統整分類結果)
* [分類結果分析](#分類結果分析)
    * [分類結果比較](#分類結果比較)
    * [預測錯誤次數](#預測錯誤次數)
* [文字雲](#文字雲)
    * [文本處理](#文本處理)
    * [繪製文字雲](#繪製文字雲)
    * [文本匯出](#文本匯出)

## 資料（內文）處理

In [1]:
import pandas as pd
# news = pd.read_csv('D://Downloads//2021大四下//大數據分析//Final Project//新聞分類統整.csv')
news = pd.read_csv('./新聞分類統整.csv')
news = news.drop(columns = ['Unnamed: 0','number'])
news = news.rename(columns = {'content': '內文'})
news.head()

,title,category,內文,內文斷詞
0,永豐餘工紙董事長換人 由邱創華出任、擬4方向衝營運成長,財經,[' ▲永豐餘工紙董事長將換成邱創華擔任。（圖／記者余弦妙攝） 記者余弦妙／台北報導 永豐...,NaN
1,趙少康臉書「表態黨主席挺朱立倫」 江啟臣幕僚緊張留言,政治,[' ▲趙少康臉書大玩愚人節哏。（圖／翻攝自趙少康臉書） 記者羅婉庭／台北報導 中...,NaN
2,兒媳暗示2024大選 川普要「挽救美國」 慘遭臉書封殺,國際,[' ▲最新民調顯示，前總統川普在2024年大選共和黨黨內提名中非常受歡迎。（圖／路...,NaN
3,掛牌58年「老牌大廠」坑殺股民！蘋果裝成奧梨仔 4萬人虧慘了,財經,[' 圖文／鏡週刊 掛牌長達58年的老牌台灣紙業公司，去年4月召開臨時股東會決定下市後，...,NaN
4,楊丞琳再演靈異片 哭完背部感受「有手輕拍」！回頭得知溫暖真相,影視與娛樂,[' 記者吳孟庭／台北報導 楊丞琳在《紅衣小女孩2》後再度演出靈異驚悚電影，於新片《靈語》...,NaN


### 正規表示法 
#### 利用 re 刪去數字、英文、符號

In [2]:
import re
new_news_list = list(news['內文'])    
for i in range(len(new_news_list)):
    new_news_list[i] = re.sub(r'[\d\WA-Za-z]+', '', new_news_list[i])

### 斷詞
#### 利用 jieba 斷詞

In [3]:
import jieba
content_piece = []
for article in new_news_list:
    seg_list = jieba.cut(article)
    content_piece.append(' '.join(seg_list))
    
news['內文斷詞'] = content_piece
news = news.rename(columns = {'內文': 'content'})
news.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 1.158 seconds.
Prefix dict has been built successfully.


,title,category,content,內文斷詞
0,永豐餘工紙董事長換人 由邱創華出任、擬4方向衝營運成長,財經,[' ▲永豐餘工紙董事長將換成邱創華擔任。（圖／記者余弦妙攝） 記者余弦妙／台北報導 永豐...,永豐餘 工紙 董事 長 將換成 邱 創華擔 任圖 記者 余弦 妙攝 記者 余弦 妙 台北 報...
1,趙少康臉書「表態黨主席挺朱立倫」 江啟臣幕僚緊張留言,政治,[' ▲趙少康臉書大玩愚人節哏。（圖／翻攝自趙少康臉書） 記者羅婉庭／台北報導 中...,趙 少康 臉書大 玩 愚人 節哏 圖翻 攝自 趙 少康 臉書 記者羅婉庭 台北 報導 中廣 ...
2,兒媳暗示2024大選 川普要「挽救美國」 慘遭臉書封殺,國際,[' ▲最新民調顯示，前總統川普在2024年大選共和黨黨內提名中非常受歡迎。（圖／路...,最新 民調 顯示 前 總 統川普 在 年 大選 共和 黨黨 內 提名 中 非常 受歡 迎圖 ...
3,掛牌58年「老牌大廠」坑殺股民！蘋果裝成奧梨仔 4萬人虧慘了,財經,[' 圖文／鏡週刊 掛牌長達58年的老牌台灣紙業公司，去年4月召開臨時股東會決定下市後，...,圖文鏡 週刊 掛牌 長 達 年 的 老牌 台灣 紙業 公司 去年 月 召開 臨時 股東會決 ...
4,楊丞琳再演靈異片 哭完背部感受「有手輕拍」！回頭得知溫暖真相,影視與娛樂,[' 記者吳孟庭／台北報導 楊丞琳在《紅衣小女孩2》後再度演出靈異驚悚電影，於新片《靈語》...,記者 吳孟庭 台北 報導 楊丞琳 在 紅衣 小女孩 後 再度 演出 靈異 驚悚電影 於 新片...


### 匯出資料
#### （含有全部資料內文和內文斷詞）

In [4]:
#news.to_csv('新聞分類統整(含斷詞).csv')
#news['category'].value_counts()

---
## 訓練模型（分類器）
**此部分將不納入報告中，為當初測試用 :)**  
https://tlyu0419.github.io/2020/04/04/Text-Classification/?fbclid=IwAR2t27jNAu-FxEQs0r2UwAmQZu_HEH0Ou9aYibQJAnvh9L9gZk1dTMcI0Fc
### 將有段詞的內文，以不同模型, min_df = 2，分別用 Count-Vector 和 TFIDF-Vector 的文字特徵處理方法，去訓練模型

### 分割資料
#### 將資料分成訓練集（0.8）和測試集（0.2），random_state = 0

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(news['內文斷詞'], news['category'], test_size = 0.2, random_state = 0)

### Count-Vector
#### 將 X 訓練集和測試集經 CountVectorizer 處理

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vect = CountVectorizer(token_pattern = r'\w{1,}', min_df = 2)
count_vect.fit(news['內文斷詞'])
counts_train = CountVectorizer(vocabulary = count_vect.vocabulary_).fit_transform(X_train)
counts_test = CountVectorizer(vocabulary = count_vect.vocabulary_).fit_transform(X_test)
counts_train

<7356x85096 sparse matrix of type '<class 'numpy.int64'>'
	with 1193298 stored elements in Compressed Sparse Row format>

#### Logistic Regression 分類器

In [7]:
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics
LR_clf = LogisticRegression(max_iter=1000, n_jobs=-1)   

In [8]:
%%time
# Maximum number of iterations taken for the solvers to converge.
LR_clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Classifier: Logistic regression; method: count vector')
LR_CV_train = LR_clf.predict(counts_train)
LR_CV_test = LR_clf.predict(counts_test)
print('Score on Train: ', metrics.accuracy_score(y_train, LR_CV_train))
print('Score on Test: ', metrics.accuracy_score(y_test, LR_CV_test))
print('='*100)

========================================  Score on Counts feature  ========================================
Classifier: Logistic regression; method: count vector
Score on Train:  1.0
Score on Test:  0.8085916258836324
Wall time: 2min 18s


#### XGBoost 分類器

In [9]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier(n_estimators=500, objective='multi:softmax', n_jobs=-1, silent=False)

In [10]:
%%time
# Maximum number of iterations taken for the solvers to converge.
xgb_clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Classifier:XGBoost; method: count vector')
xgb_CV_train = xgb_clf.predict(counts_train)
xgb_CV_test = xgb_clf.predict(counts_test)
print('Score on Train: ', metrics.accuracy_score(y_train, xgb_CV_train))
print('Score on Test: ', metrics.accuracy_score(y_test, xgb_CV_test))
print('='*100)

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:14:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:14:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
========================================  Score on Counts feature  ========================================
Classifier:XGBoost; method: count vector
Score on Train:  1.0
Score on Test:  0.8471995649809679
Wall time: 4min 21s


#### SVM linear分類器

In [11]:
from sklearn.svm import SVC
SVM_clf = SVC(kernel='linear')

In [12]:
%%time
SVM_clf.fit(counts_train, y_train)
print('='*40, ' Score on Counts feature ', '='*40)
print('Classifier: SVM(SVC=\'kernal\'); method: count vector')
SVM_CV_train = SVM_clf.predict(counts_train)
SVM_CV_test = SVM_clf.predict(counts_test)
print('Score on Train: ', metrics.accuracy_score(y_train, SVM_CV_train))
print('Score on Test: ', metrics.accuracy_score(y_test, SVM_CV_test))
print('='*100)

========================================  Score on Counts feature  ========================================
Classifier: SVM(SVC='kernal'); method: count vector
Score on Train:  1.0
Score on Test:  0.8091353996737357
Wall time: 3min 37s


---
### Tfidf-Vector
#### 將 X 訓練集和測試集經 TfidfVectorizer 處理

In [13]:
tfidf_vect = TfidfVectorizer(token_pattern = r'\w{1,}', min_df = 2)
tfidf_vect.fit(news['內文斷詞'])
tfidf_train = TfidfVectorizer(vocabulary=tfidf_vect.vocabulary_).fit_transform(X_train)
tfidf_test = TfidfVectorizer(vocabulary=tfidf_vect.vocabulary_).fit_transform(X_test)
tfidf_train

<7356x85096 sparse matrix of type '<class 'numpy.float64'>'
	with 1193298 stored elements in Compressed Sparse Row format>

#### Logistic Regression 分類器

In [14]:
%%time 
LR_clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Classifier: Logistic regression; method: tfidf')
LR_tfidf_train = LR_clf.predict(tfidf_train)
LR_tfidf_test = LR_clf.predict(tfidf_test)
print('Score on Train: ', metrics.accuracy_score(y_train, LR_tfidf_train))
print('Score on Test: ', metrics.accuracy_score(y_test, LR_tfidf_test))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Classifier: Logistic regression; method: tfidf
Score on Train:  0.8902936378466558
Score on Test:  0.768352365415987
Wall time: 54.2 s


#### XGBoost 分類器

In [15]:
%%time 
xgb_clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Classifier: XGBoost; method: tfidf')
xgb_tfidf_train = xgb_clf.predict(tfidf_train)
xgb_tfidf_test = xgb_clf.predict(tfidf_test)
print('Score on Train: ', metrics.accuracy_score(y_train, xgb_tfidf_train))
print('Score on Test: ', metrics.accuracy_score(y_test, xgb_tfidf_test))
print('='*100)

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:23:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:23:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
========================================  Score on TFIDF feature  ========================================
Classifier: XGBoost; method: tfidf
Score on Train:  1.0
Score on Test:  0.835236541598695
Wall time: 6min 7s


#### SVM linear分類器

In [16]:
%%time
SVM_clf.fit(tfidf_train, y_train)
print('='*40, ' Score on TFIDF feature ', '='*40)
print('Classifier: SVM(SVC=\'kernal\'); method: tfidf')
SVM_tfidf_train = SVM_clf.predict(tfidf_train)
SVM_tfidf_test = SVM_clf.predict(tfidf_test)
print('Score on Train: ', metrics.accuracy_score(y_train,  SVM_tfidf_train))
print('Score on Test: ', metrics.accuracy_score(y_test, SVM_tfidf_test))
print('='*100)

========================================  Score on TFIDF feature  ========================================
Classifier: SVM(SVC='kernal'); method: tfidf
Score on Train:  0.975258292550299
Score on Test:  0.8069603045133225
Wall time: 5min 34s


### 統整分類結果

In [17]:
pd.DataFrame({'Model':['Logistic', 'XGBoost', 'SVM', 'Logistic', 'XGBoost', 'SVM'],
              'Method': ['CV','CV','CV','tfidf','tfidf','tfidf'],
              'min_df': [2,2,2,2,2,2],
              'content': ['斷詞','斷詞','斷詞','斷詞','斷詞','斷詞'],
              'Train accuracy': 
              [metrics.accuracy_score(y_train, LR_CV_train),
               metrics.accuracy_score(y_train, xgb_CV_train),
               metrics.accuracy_score(y_train, SVM_CV_train),
               metrics.accuracy_score(y_train, LR_tfidf_train),
               metrics.accuracy_score(y_train, xgb_tfidf_train),
               metrics.accuracy_score(y_train, SVM_tfidf_train)],
             'Test accuracy': 
              [metrics.accuracy_score(y_test, LR_CV_test),
               metrics.accuracy_score(y_test, xgb_CV_test),
               metrics.accuracy_score(y_test, SVM_CV_test),
               metrics.accuracy_score(y_test, LR_tfidf_test),
               metrics.accuracy_score(y_test, xgb_tfidf_test),
               metrics.accuracy_score(y_test, SVM_tfidf_test)]}, 
             )


,Model,Method,min_df,content,Train accuracy,Test accuracy
0,Logistic,CV,2,斷詞,1.000000,0.808592
1,XGBoost,CV,2,斷詞,1.000000,0.847200
2,SVM,CV,2,斷詞,1.000000,0.809135
3,Logistic,tfidf,2,斷詞,0.890294,0.768352
4,XGBoost,tfidf,2,斷詞,1.000000,0.835237
5,SVM,tfidf,2,斷詞,0.975258,0.806960


---

## 分類結果分析
**此部分將不納入報告中，為當初測試用 :)**  

### 分類結果比較
#### 將每個模型針對測試集的預測結果與測試集的真實結果，和文章標題與斷詞內文一起做成DataFrame

In [18]:
compare = pd.concat([X_test, y_test], axis = 1)

clf_result = pd.DataFrame({'Logistic.CV': LR_CV_test, 'Logistic.tfidf': LR_tfidf_test, 
              'XGBoost.CV': xgb_CV_test, 'XGBoost.tfidf': xgb_tfidf_test, 
              'SVM.CV': SVM_CV_test,'SVM.tfidf': SVM_tfidf_test}, index = compare.index)

compare = pd.concat([news['title'][compare.index], compare, clf_result], axis = 1)
compare.head()
#compare.to_csv('預測結果比對.csv')

,title,內文斷詞,category,Logistic.CV,Logistic.tfidf,XGBoost.CV,XGBoost.tfidf,SVM.CV,SVM.tfidf
4226,新冠肺炎》新增281例本土，261例校正回歸！急診醫師曝一張確診者X光照，只咳一天不治身亡,指揮 中心 指揮 官陳時 中 今日 宣布 有 本土 確診例 境外 例 校正 回歸例 共 新增...,生活,生活,生活,生活,生活,生活,生活
6785,台北市24日起禁內用 侯布雄外帶滿1500元贈麵包組,台北市 宣布 自月 起 台北市 飲食店 餐廳 全面禁止 內用 只 許外 送 或 外帶 目前 ...,生活,生活,生活,生活,生活,生活,生活
2006,印度爆37萬確診新高！空地成火葬場 村民絕望求助巫醫「烙鐵驅魔」,印度 新冠 肺炎 疫情 大爆 發圖 路透 記者 郭家榮 綜合 外電 報導 印度 新冠 肺炎 ...,國際,國際,國際,國際,國際,國際,國際
5754,三級警戒延長 微風縮短營業時間「高樓層餐廳」首推外帶外送,因應 政府 延長 第三 級 警戒 包括 微風 廣場 微風 南京 微風松 高 微風 信義微風 ...,生活,生活,生活,生活,生活,生活,生活
3453,百想／《精神病》吳正世2度得最佳男配：謝謝弟弟 金秀賢眼眶濕了,記者 吳睿慈 綜合 報導 南韓 一年一度 盛典 第回 百想 藝術 大賞 日 在 首爾登場 申...,影視與娛樂,影視與娛樂,影視與娛樂,影視與娛樂,影視與娛樂,影視與娛樂,影視與娛樂


### 預測錯誤次數
#### 找出那些文章預測錯誤的次數最多，並把錯誤次數最多的文章其 index 存入 list

In [19]:
# 每個方法分錯類別的文章index
wrong_index = []
for i in compare.columns[3:]:
    wrong_index.append([j for j in range(len(compare)) if compare.iloc[j][i] != compare.iloc[j]['category']])
    
wrong_count = [j for i in wrong_index for j in i]

from collections import Counter
wrong_count = dict(Counter(wrong_count))

# https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
# import operator
max_wrong_count = max(wrong_count.items())[1]     # 用 6 個模型去分類所以最多預測錯誤的次數為 6
# 錯最多次的文章index (錯最多次為全分類器都分錯)
max_wrong_index = [index for index, times in wrong_count.items() if times == max_wrong_count]
max_wrong_index[0:5]

[18, 25, 47, 54, 62]

#### 隨便找一篇錯誤次數最多的文章，看其分類結果

In [20]:
compare.iloc[max_wrong_index[0]]

title                            一個新生兒都不能少！台灣生3胎才能申請家庭幫傭 港、星有需求就可提出
內文斷詞              文鄭 閔聲 美國 中情局 將台灣 列為 全球 生育率 最低 國家 的 調查 證明 政府 因應...
category                                                      影視與娛樂
Logistic.CV                                                      政治
Logistic.tfidf                                                   政治
XGBoost.CV                                                       國際
XGBoost.tfidf                                                    國際
SVM.CV                                                           政治
SVM.tfidf                                                        政治
Name: 9029, dtype: object

---
## 文字雲

### 文本處理
#### 將訓練集的文章內容（斷詞後），將部分詞語刪去（停用詞），並依類別做文字雲的文本
#### 本來應該要寫成迴圈，但後來太懶了，加上停用詞是手動新增，每做一次都要觀察一次，因此就寫這樣沒有再做更動了

In [21]:
train = pd.concat([y_train, X_train], axis = 1)
#list(y_train.unique())
text = ''
#for i in list(y_train.unique()[0]):
#    for j in train[train['category'] == i]['內文斷詞']:
#        text += j
i = list(y_train.unique())[14]
for j in train[train['category'] == i]['內文斷詞']:
    text += j

stopword = ['等', '在', '上', '下', '是', '與', '為', '也', '日', '月', '年', '當', '時', '當', '和', '的', '及', '於',
            '有', '對', '以', '但', '後', '前', '會', '每', '再', '或', '從', '就', '而', '更', '因', '此', '所', '以', '昨',
            '今', '天', '由', '到', '都', '已', '來', '去', '往', '這', '那', '裡', '當', '讓', '了', '至', '可', '你', '我',
            '他', '她', '它', '牠', '能', '不', '很', '還', '人', '只', '要', '此', '被', '又', '像', '好', '想', '多', '用',
            '自己', '們', '個', '輛', '把', '隻', '之', '吧', '呢', '嗎', '一', '目', '說', '沒', '表示', '才', '過', '間',
            '最', '看', '大', '起', '小', '少', '卻', '些', '樣', '得', '更', '跟', '如', '其中', '著', '廣告', '請', '繼', 
            '續', '閱讀', '提供', '記者', '報導', '同']
for i in stopword:
    text = text.replace(i, '')
    


### 繪製文字雲
#### 將處理後的文本繪製文字雲並以圖片匯出（已將匯出步驟轉為 comment）

In [22]:
# https://www.youtube.com/watch?v=HcKUU5nNmrs&ab_channel=CodingIsFunCodingIsFun
# http://120.108.221.55/profchwu/dctai/%E6%95%99%E6%9D%90/%E6%96%B7%E8%A9%9E%E8%88%87%E6%96%87%E5%AD%97%E9%9B%B2/%E6%96%B7%E8%A9%9E%E8%88%87%E6%96%87%E5%AD%97%E9%9B%B2%E6%95%99%E5%AD%B8.pdf
# https://wordcloud.timdream.org/   線上幫你用文字雲喔~~~
#text = train[train['category']=='地方']['內文斷詞'].iloc[0]
from wordcloud import  WordCloud
font_path = 'C:\\Windows\\Fonts\\simsun.ttc'
wc = WordCloud(
    background_color = 'white',
    font_path=font_path,
    height = 600, 
    width = 400
)
wc.generate(text)
#wc.to_file('wordcloud_test15.png')

### 文本匯出
#### 將處理後的文本匯出，以供線上文字雲繪製

In [23]:
# https://oxygentw.net/blog/computer/python-file-utf8-encoding/
#text_file = open("test.txt", mode = "w",encoding="utf-8")
#text_file.write(text)
#text_file.close()